In [3]:
import scanpy as sc
import scanpy.external as sce
#sc.logging.print_versions()
#sc.logging.print_memory_usage()
#sc.settings.verbosity = 2
import os,sys
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import matplotlib.ticker as mticker

In [4]:
# add the utility function folder to PATH
import sys
sys.path.append(os.path.abspath("utility_functions/"))

from rz_import_statements import *
import rz_functions as rz
import rz_fig_params # this adjust mpl.rcParams, almost nothing to import, import after scanpy to overwrite rc.Params
import rz_utility_spring as srz

python version: 3.8.8


# Load data

In [3]:
adata = sc.read_h5ad('backups/all_cells_w_dblts_batch_corrected_20021x3000_230928_15h56.h5ad') 

# overwrite obs with the most recent version
filename = 'backups/obs_info_scrub_20021x13_230928_16h03.npz'
encoding = 'latin1'

with np.load(filename,encoding=encoding, allow_pickle = True) as f:
    obs = pd.DataFrame(**f)
adata.obs = obs

In [4]:
# load intermediates I saved when preparing the SPRING plot
path1 = "/Users/karolisgoda/Library/CloudStorage/GoogleDrive-karolisgoda@gmail.com/My Drive/MF/data/SPRING_dev-master/data/" 
project_dir = path1+'CD34/'
plot_name =  'all_cells_w_dblts_spring'


params = rz.load_stuff(project_dir+plot_name+'/params.pickle')
params.keys()

dict_keys(['k', 'cell_mask', 'min_counts', 'min_cells', 'base_ix', 'num_pc', 'plot_name', 'embedding', 'gene_names_excluded', 'abundant_gene_mask', 'v_score_dict', 'nr_var_genes', 'genes_used', 'eigenvectors', 'eigenvalues', 'neighbors_fa', 'neighbors_umap', 'min_dist'])

In [5]:
cell_mask = params['cell_mask']

In [6]:
cdata = adata[cell_mask] #applying the same cell mask as used for the graph, here not necessary as all cells were used

In [7]:
G  = cdata.obsp['connectivities'] #the scanpy adjacency matrix is stored here

# Spectral clustering (choose number of cluster)

In [ ]:
# start a cell grouping dictionary:
cg = {}

start=time.time()
# slow...
for i in [500]:
    key = 'sp_cl_%s_%d'%(plot_name,i)
    cg[key] = list(srz.spec_clust(G,i).astype(str))
    print(key,(time.time()-start)/60.,'min.')

# Append result to the same SPRING plot

In [22]:
# load current color dictionary
cg0 = srz.read_cell_groupings(project_dir+plot_name+'/categorical_coloring_data.json')


# color dictionary of dictionaries
cdd = {key:value['label_colors'] for key,value in cg0.items()}


In [23]:
cg = {key:list(np.array(value).astype(str)) for key,value in cg.items()}

In [24]:
# append categorical colortrack
srz.append_cell_groupings(project_dir+plot_name,cg,colordd=cdd)

In [ ]:
cg.keys()

In [28]:
#adding for the other (UMAP) plot too
plot_name =  'all_cells_w_dblts_spring'

In [29]:
# append categorical colortrack
srz.append_cell_groupings(project_dir+plot_name,cg,colordd=cdd)

In [30]:
for key, value in cg.items():
    cdata.obs[key] = cg[key]

In [31]:
obs = cdata.obs

In [ ]:
#save obs with clusters

fname = 'backups/obs_info_scrub_%dx%d_%s'%(obs.shape[0],obs.shape[1],rz.now())
print(fname)
rz.save_df(obs,fname)